## 이전 대화를 기억하는 Memory 클래스
- langchain 참고 사이트 : https://python.langchain.com/docs/versions/migrating_memory/conversation_buffer_memory/

In [1]:
from getpass import getpass
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# LLM과 langchain의 다른 기능들을 하나로 묶어(체인 형태) 일련의 범용 작업을 처리하는 클래스
from langchain.chains import LLMChain
# LLM이 이전 대화 내을 기억할 수 있는 메모리를 구현해주는 클래스(사용하면 LLM에 질의를 넘겨줄 때 이전 대화 내용도 같이 넘겨주는 방식)
from langchain.memory import ConversationBufferMemory
# 이전 대화 중 일부만큼만 저장하는 메모리 클래스(모든 내용을 저장하지 않고 효율적으로 메모리를 관리하고 싶을 때 활용)
from langchain.memory import ConversationBufferWindowMemory

In [2]:
MY_API_KEY = getpass.getpass("OpenAI API Key :")

OpenAI API Key : ········


In [3]:
chat_model = ChatOpenAI(model="gpt-3.5-turbo",
                        api_key=MY_API_KEY
                       )

In [26]:
# input_variables : 템플릿에 필요한 변수들 선언
 # chat_histiry : 대화 기록을 저장할 변수
 # question : 사용자 질의를 저장할 변수
 # template : 실제로 모델에게 전달될 템플릿
prompt = PromptTemplate(input_variables=["chat_history", "question"],
                        template="""You are a AI assistant.
                        You are currently havung a conversation with a human.
                        Answer the quesrions.
                        chat_history:{chat_history},
                        Human:{question}
                        AI assistant:
                        """
                       )

### 1) ConversationBufferMemory

In [5]:
# ChatMessageHistory : 대화 기록 저장 클래스
from langchain.memory import ChatMessageHistory

In [6]:
history = ChatMessageHistory()

CB_memory = ConversationBufferMemory(
    chat_memory=history,
    # memory_key : 대화 내용이 지정될 key값 지정(PromptTemplate객체에 있는 "chat_history"로 지정)
    memory_key = "chat_history",
    # return_messages : 대화 기록을 메시지 객체로 반환(False의 경우 텍스트만 반환)
    return_messages=True
)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_2716\1414899723.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  CB_memory = ConversationBufferMemory(


In [8]:
llm_chain = LLMChain(
    llm=chat_model,
    prompt=prompt,
    memory=CB_memory
)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_2716\2018526355.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(


### chain에서 사용하는 응답 함수
- invoke : 복잡한 파이프라인이나 여러 구성요소와 상호작용하는 시나리오에서 유연한 응답을 생성(dict형태로 출력되며 이전 내용이 출력에 포함됨)
- predict : 단순한 시나리오에서 빠르고 간편한 텍스트 응답을 생성(이전 내용이 출력에 포함되지는 않음)

In [9]:
llm_chain.invoke("안녕?")

{'question': '안녕?',
 'chat_history': [HumanMessage(content='안녕?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={})],
 'text': '안녕하세요! 무엇을 도와드릴까요?'}

In [10]:
llm_chain.invoke("잘지내니?")

{'question': '잘지내니?',
 'chat_history': [HumanMessage(content='안녕?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='잘지내니?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='네, 제가 감정을 가지고 있는 것은 아니지만 도움이 필요하시면 언제든지 말씀해주세요. 혹시 도와드릴 것이 있나요?', additional_kwargs={}, response_metadata={})],
 'text': '네, 제가 감정을 가지고 있는 것은 아니지만 도움이 필요하시면 언제든지 말씀해주세요. 혹시 도와드릴 것이 있나요?'}

In [11]:
llm_chain.invoke("자니..?")

{'question': '자니..?',
 'chat_history': [HumanMessage(content='안녕?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='잘지내니?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='네, 제가 감정을 가지고 있는 것은 아니지만 도움이 필요하시면 언제든지 말씀해주세요. 혹시 도와드릴 것이 있나요?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='자니..?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='잠들지 않았어요. 도움이 필요하신가요?', additional_kwargs={}, response_metadata={})],
 'text': '잠들지 않았어요. 도움이 필요하신가요?'}

In [12]:
llm_chain.predict(question="너는 누구니?")

'저는 인공지능(AI) 어시스턴트입니다. 도움이 필요하신 가요?'

In [13]:
llm_chain.predict(question="어디에 사니?")

'제가 사는 곳은 인터넷 상의 데이터 센터입니다. 저는 어디든 접속이 가능하며 도움이 필요한 곳에 있습니다. 돍울이 필요하시면 언제든지 말씀해주세요.'

In [14]:
# 메모리 내용 확인
CB_memory.chat_memory

InMemoryChatMessageHistory(messages=[HumanMessage(content='안녕?', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}), HumanMessage(content='잘지내니?', additional_kwargs={}, response_metadata={}), AIMessage(content='네, 제가 감정을 가지고 있는 것은 아니지만 도움이 필요하시면 언제든지 말씀해주세요. 혹시 도와드릴 것이 있나요?', additional_kwargs={}, response_metadata={}), HumanMessage(content='자니..?', additional_kwargs={}, response_metadata={}), AIMessage(content='잠들지 않았어요. 도움이 필요하신가요?', additional_kwargs={}, response_metadata={}), HumanMessage(content='너는 누구니?', additional_kwargs={}, response_metadata={}), AIMessage(content='저는 인공지능(AI) 어시스턴트입니다. 도움이 필요하신 가요?', additional_kwargs={}, response_metadata={}), HumanMessage(content='어디에 사니?', additional_kwargs={}, response_metadata={}), AIMessage(content='제가 사는 곳은 인터넷 상의 데이터 센터입니다. 저는 어디든 접속이 가능하며 도움이 필요한 곳에 있습니다. 돍울이 필요하시면 언제든지 말씀해주세요.', additional_kwargs={}, response_metadata={})])

In [15]:
# 메모리 삭제
CB_memory.clear()

In [16]:
CB_memory.chat_memory

InMemoryChatMessageHistory(messages=[])

### 2) ConversationBufferWindowMemory
- 이전의 일정한 대화 개수(k개)만큼만 저장하는 메모리 클래스

In [31]:
history2 = ChatMessageHistory()

CBW_memory = ConversationBufferWindowMemory(chat_memory=history2,
                                            memory_key="chat_history",
                                            return_messages=True,
                                            k=3    # k는 기억할 대화 개수(window 수)
                                           )

In [32]:
llm_chain2 = LLMChain(
    llm=chat_model,
    prompt=prompt,
    memory=CBW_memory
)

In [33]:
llm_chain2.invoke({"question": "점심 메뉴 추천좀"})

{'question': '점심 메뉴 추천좀',
 'chat_history': [],
 'text': '알겠습니다. 어떤 음식을 좋아하시나요? 아시아 음식, 양식, 혹은 빵과 디저트 중에서 선호하시는 종류가 있나요? 부가적으로 알레르기나 식습관에 특별한 제약이 있을 경우 알려주시면 더 정확한 추천을 드릴 수 있습니다.'}

In [34]:
llm_chain2.invoke("느끼한 거 먹고 싶어")

{'question': '느끼한 거 먹고 싶어',
 'chat_history': [HumanMessage(content='점심 메뉴 추천좀', additional_kwargs={}, response_metadata={}),
  AIMessage(content='알겠습니다. 어떤 음식을 좋아하시나요? 아시아 음식, 양식, 혹은 빵과 디저트 중에서 선호하시는 종류가 있나요? 부가적으로 알레르기나 식습관에 특별한 제약이 있을 경우 알려주시면 더 정확한 추천을 드릴 수 있습니다.', additional_kwargs={}, response_metadata={})],
 'text': '알겠습니다. 무거운 음식을 좋아하시는군요. 그럼 스테이크나 파스타 등의 양식 음식을 추천해 드릴까요? 혹은 다른 메뉴를 원하시나요?'}

In [35]:
llm_chain2.invoke("양식보다는 한식")

{'question': '양식보다는 한식',
 'chat_history': [HumanMessage(content='점심 메뉴 추천좀', additional_kwargs={}, response_metadata={}),
  AIMessage(content='알겠습니다. 어떤 음식을 좋아하시나요? 아시아 음식, 양식, 혹은 빵과 디저트 중에서 선호하시는 종류가 있나요? 부가적으로 알레르기나 식습관에 특별한 제약이 있을 경우 알려주시면 더 정확한 추천을 드릴 수 있습니다.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='느끼한 거 먹고 싶어', additional_kwargs={}, response_metadata={}),
  AIMessage(content='알겠습니다. 무거운 음식을 좋아하시는군요. 그럼 스테이크나 파스타 등의 양식 음식을 추천해 드릴까요? 혹은 다른 메뉴를 원하시나요?', additional_kwargs={}, response_metadata={})],
 'text': '알겠습니다. 한식을 원하시군요. 한식 중에서 특별히 원하시는 음식이 있나요? 문어 숙회, 갈비찜, 불고기 등 어떤 요리를 추천해 드릴까요?'}

In [37]:
llm_chain2.predict(question="우리의 대화는 얼마나 이어질까?")

'우리의 대화는 계속될 수 있습니다. 당신이 원하는 만큼 계속 대화할 수 있습니다. 더 필요한 정보나 도움이 있으면 언제든지 말씀해주세요.'